In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pwd

'/Users/Monika/Projects'

In [3]:
df_collisions = pd.read_csv('data/collisions_orig.csv', parse_dates=["INCDTTM"])
df_locations_xy = pd.read_csv('data/location_xy.csv')

In [4]:
# Check the starting shape
df_collisions.shape

(220436, 40)

In [5]:
# Save column list to restore column order after merge
column_list = df_collisions.columns
column_list = column_list.append(pd.Index(['fe_exists']))

In [6]:
# Drop the old X,Y and replace with the ones in location_xy.csv if they exist, otherwise, will be nan
df_collisions.drop(columns=['X','Y'],inplace=True,errors='ignore')   # get these from the location_xy.csv file
df_collisions.shape

(220436, 38)

In [7]:
df_new = pd.merge(df_collisions, df_locations_xy, on='LOCATION', how='left')

In [8]:
# Do some sanity checks
df_new.shape

(220436, 41)

In [9]:
# Restore the original order of columns
df_new = df_new[column_list]

In [10]:
df_new.drop_duplicates(inplace=True)

In [12]:
df_new['fe_exists'].value_counts()

1    216059
0      4377
Name: fe_exists, dtype: int64

In [13]:
new_keys = list(df_new['INCKEY'])
col_keys = list(df_collisions['INCKEY'])
print(f"{len(new_keys)}   {len(col_keys)}")

220436   220436


In [15]:
#Dropping columns deemed unnecessary
df_new.drop(['EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'STATUS', 'REPORTNO', 'SEGLANEKEY', 'CROSSWALKKEY', 'INCKEY','COLDETKEY'],axis=1, inplace=True)

In [16]:
#to lowercase
df_new.columns = df_new.columns.str.lower()

In [17]:
df_new.columns

Index(['x', 'y', 'objectid', 'addrtype', 'intkey', 'location', 'severitycode',
       'severitydesc', 'collisiontype', 'personcount', 'pedcount',
       'pedcylcount', 'vehcount', 'injuries', 'seriousinjuries', 'fatalities',
       'incdate', 'incdttm', 'junctiontype', 'sdot_colcode', 'sdot_coldesc',
       'inattentionind', 'underinfl', 'weather', 'roadcond', 'lightcond',
       'pedrownotgrnt', 'sdotcolnum', 'speeding', 'st_colcode', 'st_coldesc',
       'hitparkedcar', 'fe_exists'],
      dtype='object')

In [31]:
print(df_new['x'].isnull().sum())
print(df_new['y'].isnull().sum())

7189
7189


In [19]:
#FE time, total_injuries and total_person_count columns
df_new["time"]=df_new['incdttm'].dt.strftime('%H:%M')
df_new["total_injuries"]=df_new['injuries'] + df_new['seriousinjuries']  + df_new['fatalities']
df_new["total_person_count"]=df_new['personcount'] + df_new['pedcount']  + df_new['pedcylcount']

In [20]:
#FE emd column for the missing 26,000
fe_emd_crit = (df_new['weather'].isnull() &
    df_new['lightcond'].isnull() &
    df_new['roadcond'].isnull() &
    df_new['collisiontype'].isnull() &
    df_new['st_coldesc'].isnull() &
    df_new['underinfl'].isnull() &
    df_new['inattentionind'].isnull() &
    df_new['speeding'].isnull() &
    df_new['pedrownotgrnt'].isnull() &
    (df_new['vehcount'] == 0))
df_new['fe_emd'] = fe_emd_crit

In [27]:
#converting incdate to datetime
df_new["incdate"] = df_new["incdate"].astype("datetime64")

### Imputing weather data by date

In [28]:
df2 = df_new.set_index('incdate') #change index to incdate 
df2['2004-01-02'] 

,x,y,objectid,addrtype,intkey,location,severitycode,severitydesc,collisiontype,personcount,...,sdotcolnum,speeding,st_colcode,st_coldesc,hitparkedcar,fe_exists,time,total_injuries,total_person_count,fe_emd
incdate,,,,,,,,,,,,,,,,,,,,,
2004-01-02,-122.286489,47.560108,2712,Block,NaN,RAINIER AVE S BETWEEN S ALASKA ST AND S ANGELI...,0,Unknown,NaN,0,...,4002069.0,NaN,NaN,NaN,N,1,00:00,0,0,True
2004-01-02,-122.344539,47.692012,4417,Intersection,37365.0,AURORA AVE N AND N 87TH ST,2,Injury Collision,Rear Ended,4,...,4002004.0,NaN,14,From same direction - both going straight - on...,N,1,00:00,1,4,False
2004-01-02,-122.317396,47.665687,4585,Block,NaN,ROOSEVELT WAY NE BETWEEN NE 50TH ST AND NE 52N...,0,Unknown,NaN,0,...,4002076.0,NaN,NaN,NaN,N,1,00:00,0,0,True
2004-01-02,-122.374250,47.631723,5034,Block,NaN,W GALER ST FLYOVER BETWEEN ELLIOTT AVE W AND M...,0,Unknown,NaN,0,...,4002019.0,NaN,NaN,NaN,N,1,00:00,0,0,True
2004-01-02,-122.322775,47.615229,5342,Block,NaN,E PINE ST BETWEEN BOYLSTON AVE AND HARVARD AVE,1,Property Damage Only Collision,Parked Car,2,...,4002008.0,NaN,32,One parked--one moving,N,1,00:00,0,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-01-02,-122.374250,47.631723,18540,Block,NaN,W GALER ST FLYOVER BETWEEN ELLIOTT AVE W AND M...,0,Unknown,NaN,0,...,4002017.0,NaN,NaN,NaN,N,1,00:00,0,0,True
2004-01-02,-122.298592,47.713600,18547,Block,NaN,LAKE CITY WAY NE BETWEEN 27TH AVE NE AND NE 11...,1,Property Damage Only Collision,Parked Car,6,...,4002050.0,NaN,32,One parked--one moving,N,1,00:00,0,6,False
2004-01-02,-122.320223,47.626145,18651,Block,NaN,10TH AVE E BETWEEN E ROY W ST AND E ALOHA ST,1,Property Damage Only Collision,Rear Ended,4,...,4002063.0,NaN,13,From same direction - both going straight - bo...,N,1,00:00,0,4,False


In [29]:
df2.loc['2004-01-02']['weather'].value_counts()

Overcast                    30
Clear                       18
Raining                      9
Snowing                      3
Unknown                      3
Sleet/Hail/Freezing Rain     3
Name: weather, dtype: int64

In [32]:
#Jeny - work in progress
#for each column, get value counts in decreasing order and take the index (value) of most common class
#weather_imputed = df2.apply(lambda x: x.fillna(x.value_counts().ind))
#df_most_common_imputed = colors.apply(lambda x: x.fillna(x.value_counts().index[0]))
#df_most_common_imputed

#df2['weather'] = df2['weather'].apply(lambda x: x.fillna(x.value_counts().index[0]))

In [21]:
df_new.columns

Index(['x', 'y', 'objectid', 'addrtype', 'intkey', 'location', 'severitycode',
       'severitydesc', 'collisiontype', 'personcount', 'pedcount',
       'pedcylcount', 'vehcount', 'injuries', 'seriousinjuries', 'fatalities',
       'incdate', 'incdttm', 'junctiontype', 'sdot_colcode', 'sdot_coldesc',
       'inattentionind', 'underinfl', 'weather', 'roadcond', 'lightcond',
       'pedrownotgrnt', 'sdotcolnum', 'speeding', 'st_colcode', 'st_coldesc',
       'hitparkedcar', 'fe_exists', 'time', 'total_injuries',
       'total_person_count', 'fe_emd'],
      dtype='object')

In [23]:
# % nulls that we have left 
df_new.isnull().sum()/len(df_new)

x                     0.032613
y                     0.032613
objectid              0.000000
addrtype              0.016817
intkey                0.675570
location              0.020759
severitycode          0.000005
severitydesc          0.000000
collisiontype         0.118638
personcount           0.000000
pedcount              0.000000
pedcylcount           0.000000
vehcount              0.000000
injuries              0.000000
seriousinjuries       0.000000
fatalities            0.000000
incdate               0.000000
incdttm               0.000000
junctiontype          0.054147
sdot_colcode          0.000005
sdot_coldesc          0.000005
inattentionind        0.863058
underinfl             0.118547
weather               0.119490
roadcond              0.119128
lightcond             0.119894
pedrownotgrnt         0.976501
sdotcolnum            0.422939
speeding              0.955198
st_colcode            0.042702
st_coldesc            0.118638
hitparkedcar          0.000000
fe_exist

In [24]:
df_new.dtypes

x                            float64
y                            float64
objectid                       int64
addrtype                      object
intkey                       float64
location                      object
severitycode                  object
severitydesc                  object
collisiontype                 object
personcount                    int64
pedcount                       int64
pedcylcount                    int64
vehcount                       int64
injuries                       int64
seriousinjuries                int64
fatalities                     int64
incdate                       object
incdttm               datetime64[ns]
junctiontype                  object
sdot_colcode                 float64
sdot_coldesc                  object
inattentionind                object
underinfl                     object
weather                       object
roadcond                      object
lightcond                     object
pedrownotgrnt                 object
s

In [26]:
#exporting to a csv, this will overwrite when more people fill in their nulls
df_new.to_csv('collisions_clean.csv')